In [1]:
import pandas as pd

In [2]:
liq = pd.read_csv("//Liqour.csv")

In [3]:
liq.head()

,Name,Ingredients,Making,Link,Base
0,Left Hand,"['1 1/2 ounces bourbon', '3/4 ounce Campari', ...",Add all ingredients into a mixing glass with i...,https://www.liquor.com/left-hand-cocktail-reci...,Bourbon Cocktails
1,Kentucky Buck,"['2 small strawberries, hulled', '3/4 ounce ri...","In a shaker, muddle the strawberries with the ...",https://www.liquor.com/kentucky-buck-cocktail-...,Bourbon Cocktails
2,Kentucky Maid,"['8 mint leaves', '2 thin cucumber slices', '3...",Place the mint leaves in a shaker and top with...,https://www.liquor.com/kentucky-maid-cocktail-...,Bourbon Cocktails
3,Improved Whiskey Cocktail,"['2 ounces bourbon or rye whiskey', '1 barspoo...",Add all ingredients into a mixing glass with i...,https://www.liquor.com/improved-whiskey-cockta...,Bourbon Cocktails
4,Mint Julep,"['8 mint leaves', '1/4 ounce simple syrup', '2...","In a Julep cup or rocks glass, lightly muddle ...",https://www.liquor.com/recipes/mint-julep/,NaN


In [4]:
liq.describe()

,Name,Ingredients,Making,Link,Base
count,849,843,843,843,312
unique,552,545,540,546,10
top,Gold Rush,"['2 ounces bourbon', '3/4 ounce lemon juice', ...","Add the bourbon, honey syrup, and lemon juice ...",https://www.liquor.com/recipes/gold-rush/,Other Cocktails
freq,10,10,10,10,41


In [320]:
liqcopy = liq.copy()

### 중복 확인 및 제거

전체 중복 확인

In [321]:
liq_ent_dup = liqcopy.duplicated(subset=None, keep=False)
len(liq_ent_dup[liq_ent_dup==True])

304

이름+재료만 중복 확인

In [322]:
liq_2_dup = liqcopy.duplicated(["Name", "Ingredients"], keep=False)
len(liq_2_dup[liq_2_dup==True])

458

이름 정보만 있고 다른 정보는 없는 칵테일 확인, 리스트 형태로 따로 모아놓기.

In [323]:
only_name = []
only_name_ind = []
for i in range(len(liqcopy)):
    if str(liqcopy["Link"][i])=='nan':
        only_name.append(liqcopy["Name"][i])
        only_name_ind.append(i)
print(only_name)
print(only_name_ind)

['1. Cinnamon Toast Crunch Shot', '2. Foolproof Fall Cocktail', '3. Apple Pie on the Rocks', '4. Fireball Pumpkin Spice Coffee', '5. Chai Fireball Whiskey Tea', '6. Cinnamon Old Fashioned']
[489, 490, 491, 492, 493, 494]


링크가 같은 칵테일은 같은 칵테일이므로 링크가 같은 칵테일들을 링크별로 딕셔너리 형태로 만들기<br>
그리고 링크가 같은 레시피들의 유니크한 링크를 Base 값이 없는 그룹, Base 값이 하나라도 있는 그룹으로 구분하기

In [324]:
liq_link_dup = liqcopy.duplicated(["Link"], keep=False)
len(liq_link_dup[liq_link_dup==True])

465

중복 링크를 가진 칵테일의 인덱스 모아주기

In [325]:
link_dup_ind = []
for i in range(len(liqcopy)):
    if liq_link_dup[i]==True:
        link_dup_ind.append(i)

그중 이름만 있는 칵테일의 인덱스는 빼주기

In [326]:
for i in only_name_ind:
    if i in link_dup_ind:
        link_dup_ind.remove(i)
    else:
        print("something wrong")

중복 링크 칵테일들의 실제 링크들을 모아주기

In [327]:
link_unique = []
for i in link_dup_ind:
    if liqcopy["Link"][i] not in link_unique:
        link_unique.append(liqcopy["Link"][i])

In [328]:
len(link_unique)

162

링크가 key이고 Base가 value인 딕셔너리 만들기 

In [329]:
link_base_dict = {}
for i in range(len(liqcopy)):
    for j in range(len(link_unique)):
        if liqcopy["Link"][i]==link_unique[j]:
            if str(liqcopy["Base"][i])!='nan':
                if link_unique[j] in list(link_base_dict.keys()):
                    link_base_dict[link_unique[j]].append(liqcopy["Base"][i])
                else:
                    link_base_dict[link_unique[j]] = [liqcopy["Base"][i]]
            else:
                link_base_dict[link_unique[j]] = []

딕셔너리의 링크(key)들을 Base(value) 유무에 따라 나눠주기

In [330]:
yes_base_dup_links = []
no_base_dup_links = []
for link in list(link_base_dict.keys()):
    if len(link_base_dict[link])!=0:
        yes_base_dup_links.append(link)
    else:
        no_base_dup_links.append(link)

In [331]:
len(yes_base_dup_links)

40

In [332]:
len(no_base_dup_links)

122

In [333]:
liqcopy["Base"].unique()

array(['Bourbon Cocktails', nan, 'Vodka Cocktails', 'Rum Cocktails',
       'Scotch Cocktails', 'Rye Whiskey Cocktails',
       'Other Whiskey Cocktails', 'Tequila & Mezcal Cocktails',
       'Cognac & Other Brandy Cocktails', 'Other Cocktails',
       'Gin Cocktails'], dtype=object)

Base가 하나라도 있는 중복 레시피들로부터 각각의 Base를 모은 것을 다시 각 레시피의 Base에 넣어주기

In [334]:
for i in range(len(liqcopy)):
    for link in yes_base_dup_links:
        if liqcopy["Link"][i]==link:
            liqcopy["Base"][i] = ", ".join(link_base_dict[link])

이제 link를 기준으로 중복 레시피를 제거하기 위해 링크 중복 레시피의 인덱스 모아주기

In [335]:
liq_link_dup_first = liqcopy.duplicated(subset="Link", keep="first")

In [336]:
liq_link_dup_first_ind = []
for i in range(len(liqcopy)):
    if liq_link_dup_first[i]==True:
        liq_link_dup_first_ind.append(i)

이름만 있는 칵테일의 인덱스는 빼주기

In [337]:
for i in only_name_ind:
    if i in liq_link_dup_first_ind:
        liq_link_dup_first_ind.remove(i)
    else:
        if i==489:
            pass
        else:
            print("something wrong")

링크 중복 제거하기

In [338]:
liqcopy.drop(liq_link_dup_first_ind, inplace=True)

In [339]:
# 확인. 6이 출력되야 정상(이름만 있는 칵테일이 6개 이므로)
liq_link_dup_after = liqcopy.duplicated(["Link"], keep=False)
len(liq_link_dup_after[liq_link_dup_after==True])

6

인덱스 재정렬

In [340]:
liqcopy.reset_index(drop=True, inplace=True)
liqcopy

,Name,Ingredients,Making,Link,Base
0,Left Hand,"['1 1/2 ounces bourbon', '3/4 ounce Campari', ...",Add all ingredients into a mixing glass with i...,https://www.liquor.com/left-hand-cocktail-reci...,Bourbon Cocktails
1,Kentucky Buck,"['2 small strawberries, hulled', '3/4 ounce ri...","In a shaker, muddle the strawberries with the ...",https://www.liquor.com/kentucky-buck-cocktail-...,Bourbon Cocktails
2,Kentucky Maid,"['8 mint leaves', '2 thin cucumber slices', '3...",Place the mint leaves in a shaker and top with...,https://www.liquor.com/kentucky-maid-cocktail-...,Bourbon Cocktails
3,Improved Whiskey Cocktail,"['2 ounces bourbon or rye whiskey', '1 barspoo...",Add all ingredients into a mixing glass with i...,https://www.liquor.com/improved-whiskey-cockta...,"Bourbon Cocktails, Rye Whiskey Cocktails"
4,Mint Julep,"['8 mint leaves', '1/4 ounce simple syrup', '2...","In a Julep cup or rocks glass, lightly muddle ...",https://www.liquor.com/recipes/mint-julep/,NaN
...,...,...,...,...,...
547,Corpse Reviver No. 2,"['Absinthe', '3/4 ounce gin', '3/4 ounce Lille...",Rinse the inside of a chilled coupe or cocktai...,https://www.liquor.com/recipes/corpse-reviver-...,NaN
548,Aviation,"['2 ounces gin', '1/2 ounce maraschino liqueur...","Add the gin, maraschino liqueur, creme de viol...",https://www.liquor.com/recipes/aviation/,NaN
549,Tom Collins,"['2 ounces gin', '1 ounce lemon juice', '1/2 o...","Add the gin, lemon juice and simple syrup to a...",https://www.liquor.com/recipes/tom-collins-2/,NaN
550,Clover Club,"['2 ounces gin', '1/2 ounce lemon juice', '1/2...","Add the gin, lemon juice, raspberry syrup and ...",https://www.liquor.com/recipes/clover-club/,NaN


다시 중복 확인 및 제거<br>
이름+재료 중복 확인

In [341]:
liq_2_dup_after = liqcopy.duplicated(["Name", "Ingredients"], keep=False)
len(liq_2_dup_after[liq_2_dup_after==True])

2

이름만 중복 확인

In [342]:
liq_name_dup = liqcopy.duplicated(["Name"], keep=False)
len(liq_name_dup[liq_name_dup==True])

2

해당 칵테일 링크 출력하기

In [343]:
for i in range(len(liqcopy)):
    if liq_name_dup[i]==True:
        print(i)
        print(liqcopy["Link"][i])

166
https://www.liquor.com/angostura-colada-cocktail-recipe-6361371
186
https://www.liquor.com/angostura-colada-cocktail-recipe-6361371?state=PREVIEW&et=1659125655486&kw=fakeAds


둘은 같은 칵테일이므로 하나를 제거하고 인덱스 재정렬하기

In [344]:
liqcopy.drop(186, inplace=True)
liqcopy.reset_index(drop=True, inplace=True)
liqcopy

,Name,Ingredients,Making,Link,Base
0,Left Hand,"['1 1/2 ounces bourbon', '3/4 ounce Campari', ...",Add all ingredients into a mixing glass with i...,https://www.liquor.com/left-hand-cocktail-reci...,Bourbon Cocktails
1,Kentucky Buck,"['2 small strawberries, hulled', '3/4 ounce ri...","In a shaker, muddle the strawberries with the ...",https://www.liquor.com/kentucky-buck-cocktail-...,Bourbon Cocktails
2,Kentucky Maid,"['8 mint leaves', '2 thin cucumber slices', '3...",Place the mint leaves in a shaker and top with...,https://www.liquor.com/kentucky-maid-cocktail-...,Bourbon Cocktails
3,Improved Whiskey Cocktail,"['2 ounces bourbon or rye whiskey', '1 barspoo...",Add all ingredients into a mixing glass with i...,https://www.liquor.com/improved-whiskey-cockta...,"Bourbon Cocktails, Rye Whiskey Cocktails"
4,Mint Julep,"['8 mint leaves', '1/4 ounce simple syrup', '2...","In a Julep cup or rocks glass, lightly muddle ...",https://www.liquor.com/recipes/mint-julep/,NaN
...,...,...,...,...,...
546,Corpse Reviver No. 2,"['Absinthe', '3/4 ounce gin', '3/4 ounce Lille...",Rinse the inside of a chilled coupe or cocktai...,https://www.liquor.com/recipes/corpse-reviver-...,NaN
547,Aviation,"['2 ounces gin', '1/2 ounce maraschino liqueur...","Add the gin, maraschino liqueur, creme de viol...",https://www.liquor.com/recipes/aviation/,NaN
548,Tom Collins,"['2 ounces gin', '1 ounce lemon juice', '1/2 o...","Add the gin, lemon juice and simple syrup to a...",https://www.liquor.com/recipes/tom-collins-2/,NaN
549,Clover Club,"['2 ounces gin', '1/2 ounce lemon juice', '1/2...","Add the gin, lemon juice, raspberry syrup and ...",https://www.liquor.com/recipes/clover-club/,NaN


다시 중복 확인 및 제거

In [345]:
liq_name_dup_after = liqcopy.duplicated(["Name"], keep=False)
len(liq_name_dup_after[liq_name_dup_after==True])

0

따로 모아놓았던 이름만있는 칵테일들에 데이터 손수 넣어주기<br>
(이름만 있는 이유: 해당 칵테일들의 레시피 정보는 다른 칵테일들과 페이지 형식이 다른 외부 링크로 넘어가야 있어서 스크래핑하지 않았음.)

In [346]:
for i in range(len(liqcopy)):
    if liqcopy["Name"][i] in only_name:
        print(i)
        print(liqcopy["Name"][i])

321
1. Cinnamon Toast Crunch Shot
322
2. Foolproof Fall Cocktail
323
3. Apple Pie on the Rocks
324
4. Fireball Pumpkin Spice Coffee
325
5. Chai Fireball Whiskey Tea
326
6. Cinnamon Old Fashioned


In [347]:
liqcopy["Name"][321] = "Cinnamon Toast Crunch Shot"
liqcopy["Ingredients"][321] = "2 oz Rum Chata, 1 oz Fireball, Cinnamon/Sugar to rim glasses (2-1 ratio for cinnamon/sugar)"
liqcopy["Making"][321] = "Rim 2 shot glasses with cinnamon/sugar. In a cocktail shaker, add ice. Pour in the Rum Chata and Fireball. Shake and strain into glasses. Enjoy! *Please drink responsibly"
liqcopy["Link"][321] = "https://thecookinchicks.com/cinnamon-toast-crunch-shots/"
liqcopy["Base"][321] = None

liqcopy["Name"][323] = "Apple Pie on the Rocks"
liqcopy["Ingredients"][323] = "1 oz Vanilla Vodka, 1 oz Fireball Whiskey, 4 oz Organic Apple Juice, Pinch of Ground Cinnamon, Brown Sugar for the rim, Grind a little finer for more successful sugaring, Optional: Cinnamon Stick for Garnish"
liqcopy["Making"][323] = "Wet the rim of your rocks glass just a little, and gently sugar the rim with the brown sugar. In a martini shaker, pour the vodka, Fireball, apple juice and ground cinnamon together with a handful of ice cubes. Shake 10 times and pour the contents into the glass without wiping off the sugar rim. Serve chilled and with a cinnamon stick, for garnish."
liqcopy["Link"][323] = "https://veganyackattack.com/2013/12/03/apple-pie-on-the-rocks/"
liqcopy["Base"][323] = "Vodka"

liqcopy["Name"][324] = "Fireball Pumpkin Spice Coffee"
liqcopy["Ingredients"][324] = "1.5 oz Fireball Cinnamon Whiskey, 1.5 oz Coffee, 4 oz Milk, 4 oz Pumpkin Puree, 1 tsp Pumpkin Pie Spice, 1 tsp Maple Syrup, Whipped Cream"
liqcopy["Making"][324] = "Place a saucepan over medium heat and pour in the pumpkin puree, pumpkin pie spice, coffee, milk, and maple syrup. Stir until hot. Add whiskey and ladle into mugs. Top with whipped cream. Garnish with caramel sauce and cinnamon sticks."
liqcopy["Link"][324] = "https://www.mainespirits.com/recipes/fireball-pumpkin-spice-coffee"
liqcopy["Base"][324] = None

# 322, 325, 326은 자세한 레시피 정보가 없었음

liqcopy.drop([322, 325, 326], inplace=True)
liqcopy.reset_index(drop=True, inplace=True)
liqcopy

,Name,Ingredients,Making,Link,Base
0,Left Hand,"['1 1/2 ounces bourbon', '3/4 ounce Campari', ...",Add all ingredients into a mixing glass with i...,https://www.liquor.com/left-hand-cocktail-reci...,Bourbon Cocktails
1,Kentucky Buck,"['2 small strawberries, hulled', '3/4 ounce ri...","In a shaker, muddle the strawberries with the ...",https://www.liquor.com/kentucky-buck-cocktail-...,Bourbon Cocktails
2,Kentucky Maid,"['8 mint leaves', '2 thin cucumber slices', '3...",Place the mint leaves in a shaker and top with...,https://www.liquor.com/kentucky-maid-cocktail-...,Bourbon Cocktails
3,Improved Whiskey Cocktail,"['2 ounces bourbon or rye whiskey', '1 barspoo...",Add all ingredients into a mixing glass with i...,https://www.liquor.com/improved-whiskey-cockta...,"Bourbon Cocktails, Rye Whiskey Cocktails"
4,Mint Julep,"['8 mint leaves', '1/4 ounce simple syrup', '2...","In a Julep cup or rocks glass, lightly muddle ...",https://www.liquor.com/recipes/mint-julep/,NaN
...,...,...,...,...,...
543,Corpse Reviver No. 2,"['Absinthe', '3/4 ounce gin', '3/4 ounce Lille...",Rinse the inside of a chilled coupe or cocktai...,https://www.liquor.com/recipes/corpse-reviver-...,NaN
544,Aviation,"['2 ounces gin', '1/2 ounce maraschino liqueur...","Add the gin, maraschino liqueur, creme de viol...",https://www.liquor.com/recipes/aviation/,NaN
545,Tom Collins,"['2 ounces gin', '1 ounce lemon juice', '1/2 o...","Add the gin, lemon juice and simple syrup to a...",https://www.liquor.com/recipes/tom-collins-2/,NaN
546,Clover Club,"['2 ounces gin', '1/2 ounce lemon juice', '1/2...","Add the gin, lemon juice, raspberry syrup and ...",https://www.liquor.com/recipes/clover-club/,NaN


In [348]:
liqcopy.describe()

,Name,Ingredients,Making,Link,Base
count,548,548,548,548,277
unique,548,548,543,548,19
top,Left Hand,"['1 1/2 ounces bourbon', '3/4 ounce Campari', ...",Add all ingredients into a shaker with ice and...,https://www.liquor.com/left-hand-cocktail-reci...,Other Cocktails
freq,1,1,5,1,39


### Ingredients 형식 맞추기

양 끝의 '['와 ']' 떼주기

In [349]:
for i in range(len(liqcopy)):
    if liqcopy["Ingredients"][i][0]=='[':
        ingredients = liqcopy["Ingredients"][i][1:-1]
        liqcopy["Ingredients"][i] = ingredients

In [350]:
# 확인
print(liqcopy["Ingredients"][1])

'2 small strawberries, hulled', '3/4 ounce rich ginger syrup (3 parts sugar to 4 parts ginger juice)', '2 ounces bourbon', '3/4 ounce lemon juice, freshly squeezed', '2 dashes Angostura bitters', 'Club soda, chilled, to top Club soda, chilled, to top (about 2-3 ounces)', 'Garnish: lemon wheel', 'Garnish: strawberry slice'


단위를 'ounce'혹은 'ounces'에서 'oz'로 바꾸기

In [351]:
for i in range(len(liqcopy)):
    ing = liqcopy["Ingredients"][i]
    new_ing = ing.replace("ounces", "oz").replace("ounce", "oz")
    liqcopy["Ingredients"][i] = new_ing

In [352]:
# 확인
print(liqcopy["Ingredients"][1])

'2 small strawberries, hulled', '3/4 oz rich ginger syrup (3 parts sugar to 4 parts ginger juice)', '2 oz bourbon', '3/4 oz lemon juice, freshly squeezed', '2 dashes Angostura bitters', 'Club soda, chilled, to top Club soda, chilled, to top (about 2-3 oz)', 'Garnish: lemon wheel', 'Garnish: strawberry slice'


Ingredients 형식상 여기까지만 형식에 맞추고 Garnish를 추출해야 해서, 다른 속성들 먼저 형식에 맞춰주고, Garnish를 추출한 후, Ingredients도 형식에 맞춰줘야 함

### Making 형식 맞추기

In [353]:
for i in range(len(liqcopy)):
    making = liqcopy["Making"][i].replace("\n", " ")
    liqcopy["Making"][i] = making

### Garnish 추출하기

In [354]:
garnish_list = [None]*548
for i in range(len(liqcopy)):
    if 'garnish:' in liqcopy["Ingredients"][i].lower():
        for ing in liqcopy["Ingredients"][i].lower().split("', '"):
            if 'garnish:' in ing:
                if ing[-1]=="'":
                    ing = ing[:-1]
                garnish_list[i] = ing[ing.index('garnish:')+8:].strip()
    else:
        if 'garnish with' in liqcopy["Making"][i].lower():
            start = liqcopy["Making"][i].lower().index("garnish with")+12
            try:
                try:
                    end = liqcopy["Making"][i].lower().index(".", start)
                except:
                    end = liqcopy["Making"][i].lower().index(",", start)
            except:
                end = None
            if end != None:
                garnish = liqcopy["Making"][i][start:end]
                garnish_list[i] = garnish
        else:
            ings = liqcopy["Ingredients"][i].split("', '")[-3:]
            maybe_garnish = []
            for ing_detail in ings:
                if "oz" not in ing_detail:
                    maybe_garnish.append(ing_detail)
            if maybe_garnish!=[]:
                print(i, maybe_garnish)

16 ['1/2 lime', "Sparkling water'"]
17 ['1 teaspoon ginger syrup', '1 sprig lavender', "lavender sprig'"]
20 ['2 dashes bitters', "orange twist'"]
27 ['1 dash bitters', '1 dash bitters', "orange twist'"]
29 ['1 splash club soda (optional)', "cinnamon stick'"]
49 ['1 dash bitters', 'dark chocolate', "dried black cherries'"]
55 ['1 teaspoon water', "orange twist'"]
67 ["nutmeg'"]
69 ["1 dash Angostura bitters'"]
70 ["Club soda'"]
72 ['2 teaspoons sugar']
75 ["grapefruit twist'"]
76 ['1 egg white']
80 ['1 teaspoon simple syrup', '2 dashes bitters', "orange twist'"]
83 ['4 star anise pods', '2 lemon peel', "6 cinnamon sticks'"]
93 ['1 1/2 tablespoons sugar']
97 ["lemon twist'"]
110 ["sugar'"]
111 ['Club soda', 'thyme sprig', "orange wheel'"]
118 ['3 pinches fresh mint']
123 ['1 teaspoon lemon zest']
127 ['1 tablespoon egg white', "grapefruit twist'"]
129 ["1 splash heavy cream'"]
138 ["2 dashes bitters'"]
140 ["1 pinch star anise powder'"]
141 ['1 barspoon lacto-fermented green tomato brin

In [355]:
garnish_coll = {20 : "orange twist", 
27 : "orange twist", 
55 : "orange twist", 
75 : "grapefruit twist", 
80 : "orange twist", 
97 : "lemon twist", 
111 : "orange wheel", 
127 : "grapefruit twist", 
148 : "lime wedge", 
209 : "lime wheel", 
249 : "lemon twist", 
262 : "lemon twist", 
263 : "lemon twist", 
271 : "orange twist", 
309 : "orange twist", 
339 : "orange twist", 
369 : "orange twist", 
492 : "orange wheel", 
527 : "orange twist"}

In [356]:
garnish_coll_key = list(garnish_coll.keys())

In [357]:
for i in range(len(garnish_list)):
    if i in garnish_coll_key:
        if garnish_list[i]!=None:
            garnish_list[i]=garnishcoll[i]

Garnish 속성 부여하고 확인하기

In [358]:
liqcopy["Garnish"]=garnish_list
liqcopy.head()

,Name,Ingredients,Making,Link,Base,Garnish
0,Left Hand,"'1 1/2 oz bourbon', '3/4 oz Campari', '3/4 oz ...",Add all ingredients into a mixing glass with i...,https://www.liquor.com/left-hand-cocktail-reci...,Bourbon Cocktails,brandied cherry
1,Kentucky Buck,"'2 small strawberries, hulled', '3/4 oz rich g...","In a shaker, muddle the strawberries with the ...",https://www.liquor.com/kentucky-buck-cocktail-...,Bourbon Cocktails,strawberry slice
2,Kentucky Maid,"'8 mint leaves', '2 thin cucumber slices', '3/...",Place the mint leaves in a shaker and top with...,https://www.liquor.com/kentucky-maid-cocktail-...,Bourbon Cocktails,mint sprig skewed through a cucumber slice
3,Improved Whiskey Cocktail,"'2 oz bourbon or rye whiskey', '1 barspoon mar...",Add all ingredients into a mixing glass with i...,https://www.liquor.com/improved-whiskey-cockta...,"Bourbon Cocktails, Rye Whiskey Cocktails",lemon twist
4,Mint Julep,"'8 mint leaves', '1/4 oz simple syrup', '2 oz ...","In a Julep cup or rocks glass, lightly muddle ...",https://www.liquor.com/recipes/mint-julep/,NaN,a mint sprig and a few drops of bitters (opti...


Ingredient 남은 형식 맞추기

' 떼주기

In [359]:
for i in range(len(liqcopy)):
    ingredient_list = liqcopy["Ingredients"][i].split(", ")
    for j in range(len(ingredient_list)):
        if ingredient_list[j][0]=="'":
            ingredient_list[j] = ingredient_list[j][1:]
        if ingredient_list[j][-1]=="'":
            ingredient_list[j] = ingredient_list[j][:-1]
    liqcopy["Ingredients"][i] = ", ".join(ingredient_list)

In [360]:
# 확인
print(liqcopy["Ingredients"][1])

2 small strawberries, hulled, 3/4 oz rich ginger syrup (3 parts sugar to 4 parts ginger juice), 2 oz bourbon, 3/4 oz lemon juice, freshly squeezed, 2 dashes Angostura bitters, Club soda, chilled, to top Club soda, chilled, to top (about 2-3 oz), Garnish: lemon wheel, Garnish: strawberry slice


### Glass 추출하기

In [361]:
glasses = []
for i in range(len(liqcopy)):
    if "glass" in liqcopy["Making"][i].lower():
        sen = liqcopy["Making"][i].lower().replace(".", "").replace(",", "")
        if ("(" in sen) and (")" in sen):
            if sen.count("(")==sen.count(")"):
                start = []
                end = []
                for l in range(len(sen)):
                    if sen[l]=="(":
                        start.append(l)
                    elif sen[l]==")":
                        end.append(l)
                for m in range(len(start)):
                    rest = sen[:start[m]]
                    change = sen[start[m]:end[m]].replace(" ", "")+sen[end[m]:]
                    sen = rest+change    
        sen = sen.split(" ")
        glass_ind = []
        for j in range(len(sen)):
            if sen[j].strip()=="glass":
                glass_ind.append(j)
        for k in glass_ind:
            if k>=3:
                if "or" in sen[k-3:k]:
                    if " ".join(sen[k-3:k]) not in glasses:
                        glasses.append(" ".join(sen[k-3:k]))
                else:
                    if sen[k-2]+" "+sen[k-1] not in glasses:
                        glasses.append(sen[k-2]+" "+sen[k-1])
            else:
                if sen[k-2]+" "+sen[k-1] not in glasses:
                    glasses.append(sen[k-2]+" "+sen[k-1])
print(glasses)

['a mixing', 'chilled coupe', 'a collins', 'double rocks', 'a rocks', 'cup or rocks', 'pack the', 'a highball', 'chilled rocks', 'the mixing', 'a pint', 'old fashioned', 'a punch', 'into the', 'collins or highball', 'in the', 'the rocks', 'the prepared', 'a large', 'stemmed beer', 'glass or collins', 'give the', 'or a rocks', 'fill the', 'to a', 'place the', 'the collins', 'over the', 'chilled cocktail', 'nora or coupe', 'a wine', 'garnish each', 'above the', 'of the', 'a cocktail', 'a coupe', 'peel over', 'coupe or cocktail', 'a 6-ounce', 'a martini', 'stemless wine', 'a tall', 'inverted cocktail', 'the cocktail', 'a hurricane', 'dip the', 'a shot', 'mug or collins', 'prepared coupe', 'goblet or hurricane', 'chilled collins', 'into a', 'chilled hurricane', 'a margarita', 'pearl diver', 'collins or hurricane', 'chilled fluted', 'and set', 'prepared rocks', 'a clay or', 'large hurricane', 'with a', 'nora or cocktail', 'cocktail or coupe', 'highball or collins', 'preheat 2', '& nora', 'c

In [362]:
len(glasses)

107

In [363]:
collection = {}
for glas in glasses:
    print(glas)
    sign = input()
    if sign=="0":
        pass
    else:
        if sign in list(collection.keys()):
            collection[sign].append(glas)
        else:
            collection[sign]=[glas]
print(collection)

# 결과는 아래 셀에 있음

a mixing


KeyboardInterrupt: Interrupted by user

0


stemmed beer는 glass or collins 앞에 붙었던 내용이어서 중복이므로 빼주기, 서로 다른 glass를 부여해줘야 하는 것은 needindex로 빼주기 

In [364]:
collection = {'Coupe Glass': ['chilled coupe', 'a coupe', 'prepared coupe', 'or a coupe'], 
              'Collins Glass': ['a collins', 'the collins', 'chilled collins', 'chilled fizz', 'or tall stemmed', 'tall collins', '8-ounce collins'], 
              'Rocks Glass': ['double rocks', 'a rocks', 'chilled rocks', 'the rocks', 'prepared rocks', 'your rocks', 'or an rocks'], 
              'Julep Cup or Rocks Glass': ['cup or rocks'], 'Highball Glass': ['a highball', 'or a highball', 'chilled highball', 'prepared highball'], 
              'Pint Glass': ['a pint', 'the pint'], 'Old Fashioned Glass': ['old fashioned'], 'Punch Glass': ['a punch'], 
              'Collins or Highball Glass': ['collins or highball'], 
              'Beer or Collins Glass': ['glass or collins'],  
              'Cocktail Glass': ['chilled cocktail', 'a cocktail', 'inverted cocktail', 'the cocktail', 'or a cocktail'], 
              'Nick & Nora or Coupe Glass': ['nora or coupe'], 'Wine Glass': ['a wine'], 
              'Coupe or Cocktail Glass': ['coupe or cocktail', 'cocktail or coupe'], 'Martini Glass': ['a martini', 'chilled martini'], 
              'Hurricane Glass': ['a hurricane', 'chilled hurricane', 'large hurricane'], 
              'Shot Glass': ['a shot', 'the shot', 'separate shot'], 'Tiki Mug or Collins Glass': ['mug or collins'], 
              'Goblet or Hurricane Glass': ['goblet or hurricane'], 'Margarita Glass': ['a margarita', 'chilled margarita', 'salted margarita'], 
              'Pearl Diver Glass': ['pearl diver'], 'Collins or Hurricane Glass': ['collins or hurricane'], 'Flute Glass': ['chilled fluted'], 
              'Clay Mug or Mug Glass': ['a clay or'], 'Nick & Nora or Cocktail Glass': ['nora or cocktail', 'glass or cocktail'], 
              'Highball or Collins Glass': ['highball or collins'], 'Nick & Nora Glass': ['& nora', 'and nora'], 
              'Old Fashioned or Rocks Glass': ['fashioned or rocks'], 'Irish Coffee Glass': ['irish coffee'], 
              'Margarita or Cocktail Glass': ['margarita or cocktail'], 'Coupe or Wine Glass': ['coupe or wine'], 
              'Cobbler Glass': ['stemmed cobbler'], 'Goblet or Wine Glass': ['goblet or wine']}

In [365]:
needindex = 'or a rocks'

In [366]:
glass_list = [None]*548
col_list = list(collection.keys())

for i in range(len(liqcopy)):
    col_list = list(collection.keys())
    glass = []
    glass_name = []
    if "glass" in liqcopy["Making"][i].lower():
        sen = liqcopy["Making"][i].lower().replace(".", "").replace(",", "")
        if ("(" in sen) and (")" in sen):
            if sen.count("(")==sen.count(")"):
                start = []
                end = []
                for l in range(len(sen)):
                    if sen[l]=="(":
                        start.append(l)
                    elif sen[l]==")":
                        end.append(l)
                for r in range(len(start)):
                    rest = sen[:start[r]]
                    change = sen[start[r]:end[r]].replace(" ", "")+sen[end[r]:]
                    sen = rest+change    
        sen = sen.split(" ")
        glass_ind = []
        for j in range(len(sen)):
            if sen[j].strip()=="glass":
                glass_ind.append(j)
        if len(glass_ind)>=2:
            for k in glass_ind:
                if k>=3:
                    if "or" in sen[k-3:k]:
                        if " ".join(sen[k-3:k]) not in glass:
                            glass.append(" ".join(sen[k-3:k]))
                    else:
                        if sen[k-2]+" "+sen[k-1] not in glass:
                            glass.append(sen[k-2]+" "+sen[k-1])
                else:
                    if sen[k-2]+" "+sen[k-1] not in glass:
                        glass.append(sen[k-2]+" "+sen[k-1])
            for p in range(len(glass)):
                for q in range(len(col_list)):
                    if glass[p] in collection[col_list[q]]:
                        if col_list[q] not in glass_name:
                            glass_name.append(col_list[q])
            two_glass = []
            one_glass = []
            must_removed = []
            for s in range(len(glass_name)):
                if " or " in glass_name[s]:
                    first, second = glass_name[s].replace(" Glass", "").split(" or ")
                    if first not in two_glass:
                        two_glass.append(first)
                    if second not in two_glass:
                        two_glass.append(second)
                else:
                    if glass_name[s] not in one_glass:
                        one_glass.append(glass_name[s])
            for m in range(len(two_glass)):
                for n in range(len(one_glass)):
                    if two_glass[m] in one_glass[n]:
                        if one_glass[n] not in must_removed:
                            must_removed.append(one_glass[n])
            if len(must_removed)!=0:
                for o in range(len(must_removed)):
                    glass_name.remove(must_removed[o])
        else:
            if glass_ind==[]:
                print('glasses or something: ', i)
            else:
                ind = glass_ind[0]
                if ind>=3:
                    if "or" in sen[ind-3:ind]:
                        if " ".join(sen[ind-3:ind]) not in glass:
                            glass=" ".join(sen[ind-3:ind])
                    else:
                        if sen[ind-2]+" "+sen[ind-1] not in glas:
                            glass=str(sen[ind-2]+" "+sen[ind-1])
                else:
                    if sen[ind-2]+" "+sen[ind-1] not in glass:
                        glass=str(sen[ind-2]+" "+sen[ind-1])
                for t in range(len(col_list)):
                    if glass in collection[col_list[t]]:
                        glass_name.append(col_list[t])
        if glass_name==[]:
            pass
        else:
            count = 0
            for u in range(len(glass_name)):
                if ' or ' in glass_name[u]:
                    count += 1
            if count<=1:
                glass_count = 0
                glass_ind = []
                for v in range(len(glass_name)):
                    if 'Glass' in glass_name[v]:
                        glass_count += 1
                        glass_ind.append(v)
                if glass_count>=2:
                    replace_count = 0
                    while replace_count<glass_ind[-1]:
                        glass_name[replace_count] = glass_name[replace_count].replace(' Glass', "")
                        replace_count += 1
                    glass_list[i]=" or ".join(glass_name)
                else:
                    glass_list[i]=" or ".join(glass_name)
            else:
                print('many or glass: ', i)

glasses or something:  114
glasses or something:  157
glasses or something:  209
glasses or something:  321
glasses or something:  346


glass가 검색되지 않은 레시피에 따로 glass 넣어주기

In [367]:
liqcopy["Making"][114]

'Fill a Moscow Mule mug (or highball glass) with ice, then add the vodka and lime juice. Top with the ginger beer. Garnish with a lime wheel.'

In [368]:
print(glass_list[114])

None


In [369]:
glass_list[114] = "Moscow Mule Mug or Highball Glass"

In [370]:
liqcopy["Making"][157]

'Add the vodka, orange liqueur and lime juice to a shaker with ice, and shake thoroughly until well-chilled. Strain into two shot glasses.'

In [371]:
print(glass_list[157])

None


In [372]:
glass_list[157] = "Shot Glass"

In [373]:
liqcopy["Making"][209]

'Serves 2 Add all ingredients into a blender with a cup and a half of ice. Pulse until mixed. Divide between two coupes or Margarita glasses and garnish each with a lime wheel.'

In [374]:
print(glass_list[209])

None


In [375]:
glass_list[209] = "Coupe or Margarita Glass"

In [376]:
liqcopy["Making"][321]

'Rim 2 shot glasses with cinnamon/sugar. In a cocktail shaker, add ice. Pour in the Rum Chata and Fireball. Shake and strain into glasses. Enjoy! *Please drink responsibly'

In [377]:
print(glass_list[321])

None


In [378]:
liqcopy["Making"][346]

'Prepare a clay cantarito cup by soaking it in cold water for 10 minutes. Rim the cantarito cup with Tajín seasoning. Add the tequila, fruit juices, and salt into a shaker with ice and shake until well-chilled. Strain into the prepared cantarito cup (or a Collins glass) over fresh ice. Top with the grapefruit soda and stir briefly and gently to combine. Garnish with a lime wedge.'

In [379]:
print(glass_list[346])

None


In [380]:
glass_list[346] = "Cantarito Cup or Collins Glass"

needindex로 빼준 레시피에 따로 glass 넣어주기

In [381]:
for i in range(len(liqcopy)):
    if 'or a rocks glass' in liqcopy["Making"][i].lower():
        print(i)
        print(liqcopy["Making"][i])
        print(glass_list[i])

42
Muddle the mint sprig in a Julep cup or a rocks glass. Add the maple beet shrub, bourbon and a small scoop of crushed ice and stir. Top with more crushed ice to fill the glass. Garnish with a mint sprig. Serve with a straw.
None
250
Add all ingredients except the beet juice into a shaker with ice and shake until chilled. Strain into a glass pipe or a rocks glass over fresh ice. Top with the beet juice. Note: While we used dry ice in the photo, we do not recommend using it in cocktails, since it's extremely dangerous to ingest. This Halloween cocktail is just as much of a treat without the tricks.
None


In [382]:
glass_list[42] = "Julep Cup or Rocks Glass"
glass_list[250] = "Rocks Glass"

glass 속성 부여하고 확인하기

In [383]:
liqcopy["Glass"] = glass_list
liqcopy.head()

,Name,Ingredients,Making,Link,Base,Garnish,Glass
0,Left Hand,"1 1/2 oz bourbon, 3/4 oz Campari, 3/4 oz sweet...",Add all ingredients into a mixing glass with i...,https://www.liquor.com/left-hand-cocktail-reci...,Bourbon Cocktails,brandied cherry,Coupe Glass
1,Kentucky Buck,"2 small strawberries, hulled, 3/4 oz rich ging...","In a shaker, muddle the strawberries with the ...",https://www.liquor.com/kentucky-buck-cocktail-...,Bourbon Cocktails,strawberry slice,Collins Glass
2,Kentucky Maid,"8 mint leaves, 2 thin cucumber slices, 3/4 oz ...",Place the mint leaves in a shaker and top with...,https://www.liquor.com/kentucky-maid-cocktail-...,Bourbon Cocktails,mint sprig skewed through a cucumber slice,Rocks Glass
3,Improved Whiskey Cocktail,"2 oz bourbon or rye whiskey, 1 barspoon marasc...",Add all ingredients into a mixing glass with i...,https://www.liquor.com/improved-whiskey-cockta...,"Bourbon Cocktails, Rye Whiskey Cocktails",lemon twist,Rocks Glass
4,Mint Julep,"8 mint leaves, 1/4 oz simple syrup, 2 oz bourb...","In a Julep cup or rocks glass, lightly muddle ...",https://www.liquor.com/recipes/mint-julep/,NaN,a mint sprig and a few drops of bitters (opti...,Julep Cup or Rocks Glass


### 기존 Base 수정 후 다른 곳에서 Base 추출하기

In [384]:
liqcopy["Base"].unique()

array(['Bourbon Cocktails', 'Bourbon Cocktails, Rye Whiskey Cocktails',
       nan, 'Vodka Cocktails', 'Gin Cocktails',
       'Vodka Cocktails, Gin Cocktails',
       'Vodka Cocktails, Tequila & Mezcal Cocktails', 'Rum Cocktails',
       'Rum Cocktails, Tequila & Mezcal Cocktails',
       'Rum Cocktails, Other Cocktails',
       'Rum Cocktails, Cognac & Other Brandy Cocktails, Gin Cocktails',
       'Scotch Cocktails', 'Rye Whiskey Cocktails',
       'Cognac & Other Brandy Cocktails',
       'Rye Whiskey Cocktails, Cognac & Other Brandy Cocktails',
       'Other Whiskey Cocktails, Cognac & Other Brandy Cocktails',
       'Other Whiskey Cocktails', None, 'Vodka',
       'Tequila & Mezcal Cocktails', 'Other Cocktails'], dtype=object)

In [385]:
base_list = [None]*548
for i in range(len(liqcopy)):
    base = []
    if liqcopy["Base"][i]=='Bourbon Cocktails':
        base.append("Whisky")
    if liqcopy["Base"][i]=='Bourbon Cocktails, Rye Whiskey Cocktails':
        base.append("Whisky")
    if liqcopy["Base"][i]=='Vodka Cocktails':
        base.append("Vodka")
    if liqcopy["Base"][i]=='Gin Cocktails':
        base.append("Gin")
    if liqcopy["Base"][i]=='Vodka Cocktails, Gin Cocktails':
        base.append("Gin")
        base.append("Vodka")
    if liqcopy["Base"][i]=='Vodka Cocktails, Tequila & Mezcal Cocktails':
        base.append("Vodka")
        base.append('t or m')
    if liqcopy["Base"][i]=='Rum Cocktails':
        base.append("Rum")
    if liqcopy["Base"][i]=='Rum Cocktails, Tequila & Mezcal Cocktails':
        base.append("Rum")
        base.append('t or m')
    if liqcopy["Base"][i]=='Rum Cocktails, Other Cocktails':
        base.append("Rum")
    if liqcopy["Base"][i]=='Rum Cocktails, Cognac & Other Brandy Cocktails, Gin Cocktails':
        base.append("Rum")
        base.append("Gin")
        base.append('c or b')
    if liqcopy["Base"][i]=='Scotch Cocktails':
        base.append("Whisky")
    if liqcopy["Base"][i]=='Rye Whiskey Cocktails':
        base.append("Whisky")
    if liqcopy["Base"][i]=='Cognac & Other Brandy Cocktails':
        base.append('c or b')
    if liqcopy["Base"][i]=='Rye Whiskey Cocktails, Cognac & Other Brandy Cocktails':
        base.append("Whisky")
        base.append('c or b')
    if liqcopy["Base"][i]=='Other Whiskey Cocktails, Cognac & Other Brandy Cocktails':
        base.append("Whisky")
        base.append('c or b')
    if liqcopy["Base"][i]=='Other Whiskey Cocktails':
        base.append("Whisky")
    if liqcopy["Base"][i]=='Vodka':
        base.append("Vodka")
    if liqcopy["Base"][i]=='Tequila & Mezcal Cocktails':
        base.append('t or m')
    
    if "vodka" in liqcopy["Ingredients"][i].lower():
        base.append("Vodka")
    if "rum" in liqcopy["Ingredients"][i].lower():
        base.append("Rum")
    if "whisky" in liqcopy["Ingredients"][i].lower() or "whiskey" in liqcopy["Ingredients"][i].lower():
        base.append("Whisky")
    if "tequila" in liqcopy["Ingredients"][i].lower() or "tekila" in liqcopy["Ingredients"][i].lower() or "teqila" in liqcopy["Ingredients"][i].lower():
        base.append("Tequila")
    if "mezcal" in liqcopy["Ingredients"][i].lower():
        base.append("Mezcal")
    if "gin" in liqcopy["Ingredients"][i].lower():
        base.append("Gin")
    if "cognac" in liqcopy["Ingredients"][i].lower():
        base.append("Cognac")
    if "brandy" in liqcopy["Ingredients"][i].lower():
        base.append("Brandy")
    if "wine" in liqcopy["Ingredients"][i].lower():
        base.append("Wine")
    if "liquor" in liqcopy["Ingredients"][i].lower():
        base.append("Liquor")
    if "beer" in liqcopy["Ingredients"][i].lower():
        base.append("Beer")
    if " ale," in liqcopy["Ingredients"][i].lower() or " ale." in liqcopy["Ingredients"][i].lower() or " ale " in liqcopy["Ingredients"][i].lower():
        base.append("Ale")
    if "champagne" in liqcopy["Ingredients"][i].lower():
        base.append("Champagne")
    
    base = list(set(base))
    if base==[]:
        pass
    else:
        if 't or m' in base:
            if "Tequila" in base or "Mezcal" in base:
                base.remove('t or m')
            else:
                base.remove('t or m')
                base.append("Tequila & Mezcal")
        if 'c or b' in base:
            if "Cognac" in base or "Brandy" in base:
                base.remove('c or b')
            else:
                base.remove('c or b')
                base.append("Cognac & Brandy")
        base_list[i] = ", ".join(base)

4
5
7
8
9
12
15
16
22
23
25
26
29
32
37
39
40
42
45
60
61
63
65
68
70
71
73
74
75
77
81
84
86
210
237
238
239
240
241
289
295
309
376
428
432
451
452
453
454
455
456
457
459
460
461
465
468
471
473
474
476
477
478
479
480
481
482
484
485
487
489
491
492
493
494
495
496
497
498
516
524


Type 속성 부여하고 확인하기

In [386]:
liqcopy["Base"]=base_list
liqcopy.head()

,Name,Ingredients,Making,Link,Base,Garnish,Glass
0,Left Hand,"1 1/2 oz bourbon, 3/4 oz Campari, 3/4 oz sweet...",Add all ingredients into a mixing glass with i...,https://www.liquor.com/left-hand-cocktail-reci...,Whisky,brandied cherry,Coupe Glass
1,Kentucky Buck,"2 small strawberries, hulled, 3/4 oz rich ging...","In a shaker, muddle the strawberries with the ...",https://www.liquor.com/kentucky-buck-cocktail-...,"Gin, Whisky",strawberry slice,Collins Glass
2,Kentucky Maid,"8 mint leaves, 2 thin cucumber slices, 3/4 oz ...",Place the mint leaves in a shaker and top with...,https://www.liquor.com/kentucky-maid-cocktail-...,Whisky,mint sprig skewed through a cucumber slice,Rocks Glass
3,Improved Whiskey Cocktail,"2 oz bourbon or rye whiskey, 1 barspoon marasc...",Add all ingredients into a mixing glass with i...,https://www.liquor.com/improved-whiskey-cockta...,Whisky,lemon twist,Rocks Glass
4,Mint Julep,"8 mint leaves, 1/4 oz simple syrup, 2 oz bourb...","In a Julep cup or rocks glass, lightly muddle ...",https://www.liquor.com/recipes/mint-julep/,None,a mint sprig and a few drops of bitters (opti...,Julep Cup or Rocks Glass


### 따로 스크랩했던 Method, Type, Flavor 속성의 속성값과 추출한 속성값 부여하기

따로 스크랩한 데이터<br>
["Type"(classic, modern classic, tiki&tropical, coffee&dessert, shot&shooters, punches, nonalcoholic, on the rocks, neat, hot, frozen&blended), <br>
"Method"(shake, build, stir, up), <br>
"Flavor"(bitter, sweet, savory, sour, spicy, fruity, smoky, herbaceous)]

In [387]:
liq_type_classic = pd.read_csv("//Liqour_type_Classics.csv")
liq_type_modern_classic = pd.read_csv("//Liqour_type_Modern Classics.csv")
liq_type_tiki_tropical = pd.read_csv("//Liqour_type_Tiki & Tropical Cocktails.csv")
liq_type_coffee_dessert = pd.read_csv("//Liqour_type_Coffee & Dessert Cocktails.csv")
liq_type_shot_shooter = pd.read_csv("//Liqour_type_Shots & Shooters.csv")
liq_type_punch = pd.read_csv("//Liqour_type_Punches.csv")
liq_type_nonalcoholic = pd.read_csv("//Liqour_type_Nonalcoholic Drinks.csv")
liq_type_rocks = pd.read_csv("//Liqour_style_Cocktails On the Rocks.csv")
liq_type_neat = pd.read_csv("//Liqour_style_Neat Cocktails.csv")
liq_type_hot = pd.read_csv("//Liqour_style_Heated Cocktails.csv")
liq_type_frozen_blended = pd.read_csv("//Liqour_style_Frozen Blended Cocktails.csv")
liq_type_up = pd.read_csv("//Liqour_style_Up Cocktails.csv")
classic_list = list(liq_type_classic["Name"])
modern_classic_list = list(liq_type_modern_classic["Name"])
tiki_tropical_list = list(liq_type_tiki_tropical["Name"])
coffee_dessert_list = list(liq_type_coffee_dessert["Name"])
shooter_list = list(liq_type_shot_shooter["Name"])
punch_list = list(liq_type_punch["Name"])
nonalcoholic_list = list(liq_type_nonalcoholic["Name"])
rocks_list = list(liq_type_rocks["Name"])
neat_list = list(liq_type_neat["Name"])
hot_list = list(liq_type_hot["Name"])
frozen_blended_list = list(liq_type_frozen_blended["Name"])
up_list = list(liq_type_up["Name"])

liq_mothod_build1 = pd.read_csv("//Liqour_style_Cocktails Built in Glass.csv")
liq_mothod_build2 = pd.read_csv("//Liqour_style_Poured Cocktails.csv")
liq_mothod_shake = pd.read_csv("//Liqour_style_Shaken Cocktails.csv")
liq_mothod_stir = pd.read_csv("//Liqour_style_Stirred Cocktails.csv")
build1_list = list(liq_mothod_build1["Name"])
build2_list = list(liq_mothod_build2["Name"])
shake_list = list(liq_mothod_shake["Name"])
stir_list = list(liq_mothod_stir["Name"])

liq_flavor_bitter = pd.read_csv("//Liqour_flavor_Bitter.csv")
liq_flavor_sweet = pd.read_csv("//Liqour_flavor_Sweet.csv")
liq_flavor_savory = pd.read_csv("//Liqour_flavor_Savory.csv")
liq_flavor_sour = pd.read_csv("//Liqour_flavor_Sour.csv")
liq_flavor_spicy = pd.read_csv("//Liqour_flavor_Spicy.csv")
liq_flavor_fruity = pd.read_csv("//Liqour_flavor_Fruity.csv")
liq_flavor_smoky = pd.read_csv("//Liqour_flavor_Smoky.csv")
liq_flavor_herbaceous = pd.read_csv("//Liqour_flavor_Herbaceous.csv")
bitter_list = list(liq_flavor_bitter["Name"])
sweet_list = list(liq_flavor_sweet["Name"])
savory_list = list(liq_flavor_savory["Name"])
sour_list = list(liq_flavor_sour["Name"])
spicy_list = list(liq_flavor_spicy["Name"])
fruity_list = list(liq_flavor_fruity["Name"])
smoky_list = list(liq_flavor_smoky["Name"])
herbaceous_list = list(liq_flavor_herbaceous["Name"])

In [388]:
liqcopy["Glass"].unique()

array(['Coupe Glass', 'Collins Glass', 'Rocks Glass',
       'Julep Cup or Rocks Glass', 'Highball Glass', 'Pint Glass',
       'Old Fashioned Glass', 'Punch Glass', None,
       'Collins or Highball Glass', 'Beer or Collins Glass',
       'Cocktail Glass', 'Nick & Nora or Coupe Glass', 'Wine Glass',
       'Coupe or Cocktail Glass', 'Martini Glass',
       'Moscow Mule Mug or Highball Glass', 'Hurricane Glass',
       'Shot Glass', 'Tiki Mug or Collins Glass',
       'Goblet or Hurricane Glass', 'Margarita Glass',
       'Pearl Diver Glass', 'Collins or Hurricane Glass',
       'Coupe or Margarita Glass', 'Flute Glass', 'Clay Mug or Mug Glass',
       'Nick & Nora or Cocktail Glass', 'Highball or Collins Glass',
       'Nick & Nora Glass', 'Old Fashioned or Rocks Glass',
       'Rocks or Cocktail Glass', 'Shot or Pint Glass',
       'Rocks or Highball Glass', 'Irish Coffee Glass',
       'Cantarito Cup or Collins Glass', 'Margarita or Cocktail Glass',
       'Coupe or Wine Glass', 'Co

잠깐 glass에서 이상한 값이 발견되어서 수정하기<br>
: shot과 pint는 or로 묶일 수 없는 glass인데 'Shot or Pint Glass'과 'Pint or Shot Glass'가 있다.

In [389]:
for i in range(len(liqcopy)):
    if liqcopy["Glass"][i]=='Shot or Pint Glass' or liqcopy["Glass"][i]=='Pint or Shot Glass':
        print(i)
        print(liqcopy["Glass"][i])
        print(liqcopy["Making"][i])

297
Shot or Pint Glass
Pour the whiskey into a shot glass. Fill a pint glass halfway with beer. Drop the shot glass into the beer.
342
Shot or Pint Glass
Add the Baileys and whiskey into a shot glass, pouring slowly to create a layered effect. Drop the shot into a pint glass filled half to three-quarters with the Guinness. Drink immediately.
484
Pint or Shot Glass
Pour the energy drink into a pint glass. Pour the Jägermeister into a shot glass. Drop the shot glass filled with Jägermeister into the pint glass.


In [390]:
liqcopy["Glass"][297]='Shot and Pint Glass'
liqcopy["Glass"][342]='Shot and Pint Glass'
liqcopy["Glass"][484]='Pint and Shot Glass'

수정 완료

In [391]:
shooter_glasses = ['Shot Glass']
short_glasses = ['Coupe Glass', 'Cocktail Glass', 'Coupe or Cocktail Glass', 'Martini Glass', 'Cocktail or Coupe Glass']
long_glasses = ['Collins Glass', 'Highball Glass', 'Collins or Highball Glass', 'Hurricane Glass', 'Collins or Hurricane Glass', 'Highball or Collins Glass']

In [394]:
type_list = [None]*548
method_list = [None]*548
flavor_list = [None]*548

for i in range(len(liqcopy)):
    types = []
    methods = []
    flavors = []
    if liqcopy["Name"][i] in classic_list:
        types.append("Classic")
    if liqcopy["Name"][i] in modern_classic_list:
        if "Classic" in types:
            types.remove("Classic")
        types.append("Modern Classic")
    if liqcopy["Name"][i] in tiki_tropical_list:
        types.append("Tiki & Tropical")
    if liqcopy["Name"][i] in coffee_dessert_list:
        types.append("Coffee & Dessert")
    if liqcopy["Name"][i] in shooter_list:
        types.append("Shooter")
    if liqcopy["Name"][i] in punch_list:
        types.append("Punch")
    if liqcopy["Name"][i] in nonalcoholic_list:
        types.append("Non-Alcoholic")
#    if liqcopy["Name"] in rocks_list:
#        types.append("")
    if liqcopy["Name"][i] in neat_list:
        types.append("Neat")
    if liqcopy["Name"][i] in hot_list:
        types.append("Hot")
    if liqcopy["Name"][i] in frozen_blended_list:
        types.append("Frozen/Blended")
        methods.append("Blend")
    if liqcopy["Name"][i] in up_list:
        types.append("Up")
        
    if liqcopy["Glass"][i] in shooter_glasses:
        if "Shooter" not in types:
            types.append("Shooter")
    if liqcopy["Glass"][i] in short_glasses:
        types.append("Short")
    if liqcopy["Glass"][i] in long_glasses:
        types.append("Long")
        
    if "shake" in liqcopy["Making"][i].lower():
        methods.append("Shake")
    if "stir" in liqcopy["Making"][i].lower() or "mixing glass" in liqcopy["Making"][i].lower():
        methods.append("Stir")
    if "combine" in liqcopy["Making"][i].lower() or "build" in liqcopy["Making"][i].lower():
        methods.append("Build")
    if "float" in liqcopy["Making"][i].lower() or "layer" in liqcopy["Making"][i].lower():
        methods.append("Float")
    if "blend" in liqcopy["Making"][i].lower():
        if "Blend" not in methods:
            methods.append("Blend")
    if "strain" in liqcopy["Making"][i].lower():
        methods.append("Strain")

    if liqcopy["Name"][i] in shake_list:
        if "Shake" not in methods:
            methods.append("Shake")
    if liqcopy["Name"][i] in stir_list:
        if "Stir" not in methods:
            methods.append("Stir")
    if liqcopy["Name"][i] in build1_list:
        if "Build" not in methods:
            if "Shake" in methods:
                methods.remove("Shake")
            methods.append("Build")
        else:
            if "Shake" in methods:
                methods.remove("Shake")
    if liqcopy["Name"][i] in build2_list:
        if "Build" not in methods:
            if "Strain" in methods:
                methods.remove("Strain")
            methods.append("Build")
        else:
            if "Strain" in methods:
                methods.remove("Strain")
    
    if liqcopy["Name"][i] in bitter_list:
        flavors.append("Bitter")
    if liqcopy["Name"][i] in sweet_list:
        flavors.append("Sweet")
    if liqcopy["Name"][i] in savory_list:
        flavors.append("Savory")
    if liqcopy["Name"][i] in sour_list:
        flavors.append("Sour")
    if liqcopy["Name"][i] in spicy_list:
        flavors.append("Spicy")
    if liqcopy["Name"][i] in fruity_list:
        flavors.append("Fruity")
    if liqcopy["Name"][i] in smoky_list:
        flavors.append("Smoky")
    if liqcopy["Name"][i] in herbaceous_list:
        flavors.append("Herbaceous")
    
    if types==[]:
        pass
    else:
        type_list[i] = ", ".join(types)
        
    if methods==[]:
        pass
    else:
        method_list[i] = ", ".join(methods)
        
    if flavors==[]:
        pass
    else:
        flavor_list[i] = " ,".join(flavors)

Type 속성 부여하고 확인하기

In [395]:
liqcopy["Type"]=type_list
liqcopy.head()

,Name,Ingredients,Making,Link,Base,Garnish,Glass,Type
0,Left Hand,"1 1/2 oz bourbon, 3/4 oz Campari, 3/4 oz sweet...",Add all ingredients into a mixing glass with i...,https://www.liquor.com/left-hand-cocktail-reci...,Whisky,brandied cherry,Coupe Glass,Short
1,Kentucky Buck,"2 small strawberries, hulled, 3/4 oz rich ging...","In a shaker, muddle the strawberries with the ...",https://www.liquor.com/kentucky-buck-cocktail-...,"Gin, Whisky",strawberry slice,Collins Glass,Long
2,Kentucky Maid,"8 mint leaves, 2 thin cucumber slices, 3/4 oz ...",Place the mint leaves in a shaker and top with...,https://www.liquor.com/kentucky-maid-cocktail-...,Whisky,mint sprig skewed through a cucumber slice,Rocks Glass,None
3,Improved Whiskey Cocktail,"2 oz bourbon or rye whiskey, 1 barspoon marasc...",Add all ingredients into a mixing glass with i...,https://www.liquor.com/improved-whiskey-cockta...,Whisky,lemon twist,Rocks Glass,Classic
4,Mint Julep,"8 mint leaves, 1/4 oz simple syrup, 2 oz bourb...","In a Julep cup or rocks glass, lightly muddle ...",https://www.liquor.com/recipes/mint-julep/,None,a mint sprig and a few drops of bitters (opti...,Julep Cup or Rocks Glass,None


Method 속성 부여하고 확인하기

In [397]:
liqcopy["Method"]=method_list
liqcopy.head()

,Name,Ingredients,Making,Link,Base,Garnish,Glass,Type,Method
0,Left Hand,"1 1/2 oz bourbon, 3/4 oz Campari, 3/4 oz sweet...",Add all ingredients into a mixing glass with i...,https://www.liquor.com/left-hand-cocktail-reci...,Whisky,brandied cherry,Coupe Glass,Short,"Stir, Strain"
1,Kentucky Buck,"2 small strawberries, hulled, 3/4 oz rich ging...","In a shaker, muddle the strawberries with the ...",https://www.liquor.com/kentucky-buck-cocktail-...,"Gin, Whisky",strawberry slice,Collins Glass,Long,"Shake, Strain"
2,Kentucky Maid,"8 mint leaves, 2 thin cucumber slices, 3/4 oz ...",Place the mint leaves in a shaker and top with...,https://www.liquor.com/kentucky-maid-cocktail-...,Whisky,mint sprig skewed through a cucumber slice,Rocks Glass,None,"Shake, Strain"
3,Improved Whiskey Cocktail,"2 oz bourbon or rye whiskey, 1 barspoon marasc...",Add all ingredients into a mixing glass with i...,https://www.liquor.com/improved-whiskey-cockta...,Whisky,lemon twist,Rocks Glass,Classic,"Stir, Strain"
4,Mint Julep,"8 mint leaves, 1/4 oz simple syrup, 2 oz bourb...","In a Julep cup or rocks glass, lightly muddle ...",https://www.liquor.com/recipes/mint-julep/,None,a mint sprig and a few drops of bitters (opti...,Julep Cup or Rocks Glass,None,Stir


Flavor 속성 부여하고 확인하기

In [398]:
liqcopy["Flavor"]=flavor_list
liqcopy.head()

,Name,Ingredients,Making,Link,Base,Garnish,Glass,Type,Method,Flavor
0,Left Hand,"1 1/2 oz bourbon, 3/4 oz Campari, 3/4 oz sweet...",Add all ingredients into a mixing glass with i...,https://www.liquor.com/left-hand-cocktail-reci...,Whisky,brandied cherry,Coupe Glass,Short,"Stir, Strain",None
1,Kentucky Buck,"2 small strawberries, hulled, 3/4 oz rich ging...","In a shaker, muddle the strawberries with the ...",https://www.liquor.com/kentucky-buck-cocktail-...,"Gin, Whisky",strawberry slice,Collins Glass,Long,"Shake, Strain",None
2,Kentucky Maid,"8 mint leaves, 2 thin cucumber slices, 3/4 oz ...",Place the mint leaves in a shaker and top with...,https://www.liquor.com/kentucky-maid-cocktail-...,Whisky,mint sprig skewed through a cucumber slice,Rocks Glass,None,"Shake, Strain",Herbaceous
3,Improved Whiskey Cocktail,"2 oz bourbon or rye whiskey, 1 barspoon marasc...",Add all ingredients into a mixing glass with i...,https://www.liquor.com/improved-whiskey-cockta...,Whisky,lemon twist,Rocks Glass,Classic,"Stir, Strain",None
4,Mint Julep,"8 mint leaves, 1/4 oz simple syrup, 2 oz bourb...","In a Julep cup or rocks glass, lightly muddle ...",https://www.liquor.com/recipes/mint-julep/,None,a mint sprig and a few drops of bitters (opti...,Julep Cup or Rocks Glass,None,Stir,None


카테고리 순서 형식에 맞게 정렬하고 확인하기

In [399]:
liqcopy = liqcopy[["Name", "Ingredients", "Making", "Link", "Glass", "Garnish", "Base", "Method", "Type", "Flavor"]]
liqcopy.head()

,Name,Ingredients,Making,Link,Glass,Garnish,Base,Method,Type,Flavor
0,Left Hand,"1 1/2 oz bourbon, 3/4 oz Campari, 3/4 oz sweet...",Add all ingredients into a mixing glass with i...,https://www.liquor.com/left-hand-cocktail-reci...,Coupe Glass,brandied cherry,Whisky,"Stir, Strain",Short,None
1,Kentucky Buck,"2 small strawberries, hulled, 3/4 oz rich ging...","In a shaker, muddle the strawberries with the ...",https://www.liquor.com/kentucky-buck-cocktail-...,Collins Glass,strawberry slice,"Gin, Whisky","Shake, Strain",Long,None
2,Kentucky Maid,"8 mint leaves, 2 thin cucumber slices, 3/4 oz ...",Place the mint leaves in a shaker and top with...,https://www.liquor.com/kentucky-maid-cocktail-...,Rocks Glass,mint sprig skewed through a cucumber slice,Whisky,"Shake, Strain",None,Herbaceous
3,Improved Whiskey Cocktail,"2 oz bourbon or rye whiskey, 1 barspoon marasc...",Add all ingredients into a mixing glass with i...,https://www.liquor.com/improved-whiskey-cockta...,Rocks Glass,lemon twist,Whisky,"Stir, Strain",Classic,None
4,Mint Julep,"8 mint leaves, 1/4 oz simple syrup, 2 oz bourb...","In a Julep cup or rocks glass, lightly muddle ...",https://www.liquor.com/recipes/mint-julep/,Julep Cup or Rocks Glass,a mint sprig and a few drops of bitters (opti...,None,Stir,None,None


데이터 저장하기

In [400]:
liqcopy.to_csv("//Liquor_ready.csv", encoding="utf-8-sig", index=False)

Liquor 데이터 전처리 완료